In [6]:
import numpy as np
import cv2
import time
import serial

class STM32BallTracker:
    def __init__(self, camera_source=0, stm32_port='COM5', baudrate=115200):
        """
        STM32 Ball Tracker - Manuel dairesel platform seçimi ile beyaz top takibi
        """
        self.camera_source = camera_source
        self.stm32_port = stm32_port
        self.baudrate = baudrate
        self.ser = None

        # Ball detection parametreleri
        self.min_area = 100
        self.max_area = 2000
        self.min_circularity = 0.5

        # Dairesel platform kalibrasyonu
        self.calibrated = False
        self.platform_center = None
        self.platform_radius = None
        self.platform_size = 16
        
        # Manuel seçim için
        self.selected_points = []
        self.temp_frame = None

        self.running = False
        self.last_sent_coords = None
        
        # Hız optimizasyonları
        self.min_coord_change = 0.1  # 0.5 cm minimum değişim (daha hassas takip)
        self.send_interval = 0.01  # 10ms minimum gönderim aralığı (100Hz)
        self.last_send_time = 0

    def init_stm32_connection(self):
        """STM32 seri port bağlantısı - Hız optimizasyonlu"""
        try:
            self.ser = serial.Serial(
                port=self.stm32_port,
                baudrate=self.baudrate,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                stopbits=serial.STOPBITS_ONE,
                timeout=0.001,  # Çok kısa timeout (1ms)
                write_timeout=0.001  # Yazma timeout
            )
            # Buffer boyutlarını optimize et
            self.ser.reset_input_buffer()
            self.ser.reset_output_buffer()
            time.sleep(0.5)
            print(f"✓ STM32 connected: {self.stm32_port} @ {self.baudrate} baud")
            return True
        except Exception as e:
            print(f"✗ STM32 connection error: {e}")
            return False

    def mouse_callback(self, event, x, y, flags, param):
        """Fare ile daire seçimi - merkez ve kenar noktası"""
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(self.selected_points) < 2:
                self.selected_points.append([x, y])
                
                if len(self.selected_points) == 1:
                    print(f"✓ Merkez: ({x}, {y})")
                    cv2.circle(self.temp_frame, (x, y), 8, (0, 255, 0), -1)
                    cv2.putText(self.temp_frame, "CENTER", 
                              (x+15, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                elif len(self.selected_points) == 2:
                    center = self.selected_points[0]
                    edge = self.selected_points[1]
                    radius = int(np.sqrt((edge[0] - center[0])**2 + (edge[1] - center[1])**2))
                    
                    print(f"✓ Kenar: ({x}, {y})")
                    print(f"✓ Yarıçap: {radius} piksel")
                    
                    cv2.circle(self.temp_frame, tuple(center), radius, (0, 255, 0), 2)
                    cv2.circle(self.temp_frame, (x, y), 8, (0, 255, 0), -1)
                    cv2.line(self.temp_frame, tuple(center), (x, y), (0, 255, 0), 2)
                    cv2.putText(self.temp_frame, "EDGE", 
                              (x+15, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    
                    print("✅ Daire seçildi! SPACE'e basın veya pencereyi kapatın...")

    def manual_platform_selection(self, frame):
        """Manuel olarak dairesel platform seç"""
        print("\n🖱  MANUEL DAİRESEL KALIBRASYON")
        print("=" * 50)
        print("1. Platformun MERKEZİNE tıklayın")
        print("2. Platformun KENARINDA bir noktaya tıklayın")
        print("3. Seçim tamamlandıktan sonra SPACE basın")
        print("4. Yeniden başlamak için R basın")
        print("=" * 50)

        self.temp_frame = frame.copy()
        self.selected_points = []

        cv2.namedWindow('Platform Seçimi')
        cv2.setMouseCallback('Platform Seçimi', self.mouse_callback)

        while True:
            cv2.imshow('Platform Seçimi', self.temp_frame)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord(' ') and len(self.selected_points) == 2:
                break
            elif key == ord('r'):
                self.selected_points = []
                self.temp_frame = frame.copy()
                print("🔄 Seçim sıfırlandı, tekrar seçin...")

        cv2.destroyWindow('Platform Seçimi')
        
        if len(self.selected_points) == 2:
            center = self.selected_points[0]
            edge = self.selected_points[1]
            radius = np.sqrt((edge[0] - center[0])**2 + (edge[1] - center[1])**2)
            return (center, radius)
        return None

    def pixel_to_platform_coords(self, pixel_point):
        """Piksel koordinatını platform koordinatına dönüştür (0-16 arası)"""
        if not self.calibrated or self.platform_center is None or self.platform_radius is None:
            return None

        dx = pixel_point[0] - self.platform_center[0]
        dy = pixel_point[1] - self.platform_center[1]

        physical_radius = self.platform_size / 2  # 8 cm
        
        x_cm = (dx / self.platform_radius) * physical_radius + physical_radius
        y_cm = (dy / self.platform_radius) * physical_radius + physical_radius

        # 0-16 arasına sınırla
        x_cm = max(0, min(self.platform_size, x_cm))
        y_cm = max(0, min(self.platform_size, y_cm))

        return (x_cm, y_cm)

    def send_coordinates_to_stm32(self, platform_coords):
        """Platform koordinatlarını STM32'ye hızlı gönder"""
        if not self.ser or not self.ser.is_open or platform_coords is None:
            return False

        # Zaman bazlı throttling
        current_time = time.time()
        if current_time - self.last_send_time < self.send_interval:
            return False

        x_cm = int(platform_coords[0] * 10)
        y_cm = int(platform_coords[1] * 10)
        
        # Minimum değişim kontrolü (daha hassas)
        if self.last_sent_coords is not None:
            last_x, last_y = self.last_sent_coords
            dx = abs(x_cm - last_x) / 10.0
            dy = abs(y_cm - last_y) / 10.0
            if dx < self.min_coord_change and dy < self.min_coord_change:
                return False

        try:
            # Sabit genişlikte format: <xxx,yyy> (3 haneli)
            data = f"<{x_cm:03d},{y_cm:03d}>\n"
            self.ser.write(data.encode('utf-8'))
            
            self.last_sent_coords = (x_cm, y_cm)
            self.last_send_time = current_time
            
            print(f"STM32 <- <{x_cm:03d},{y_cm:03d}> [{int(1/(current_time - self.last_send_time + 0.001))} Hz]")
            return True
        except Exception as e:
            print(f"STM32 send error: {e}")
            return False

    def detect_ball(self, frame):
        """Beyaz top algılama - Optimize edilmiş"""
        # Daha küçük boyuta resize et (işlem hızı için)
        small_frame = cv2.resize(frame, (320, 240))
        scale_x = frame.shape[1] / 320
        scale_y = frame.shape[0] / 240
        
        hsv = cv2.cvtColor(small_frame, cv2.COLOR_BGR2HSV)
        white_mask = cv2.inRange(hsv, (0, 0, 200), (180, 50, 255))

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        ball_center = None
        best_contour = None

        for contour in contours:
            area = cv2.contourArea(contour)
            scaled_min_area = self.min_area / (scale_x * scale_y)
            scaled_max_area = self.max_area / (scale_x * scale_y)
            
            if scaled_min_area < area < scaled_max_area:
                perimeter = cv2.arcLength(contour, True)
                if perimeter > 0:
                    circularity = 4 * np.pi * area / (perimeter * perimeter)
                    if circularity > self.min_circularity:
                        M = cv2.moments(contour)
                        if M["m00"] != 0:
                            cx = int((M["m10"] / M["m00"]) * scale_x)
                            cy = int((M["m01"] / M["m00"]) * scale_y)
                            ball_center = (cx, cy)
                            # Contour'u orijinal boyuta ölçekle
                            best_contour = (contour * [scale_x, scale_y]).astype(np.int32)
                            break

        # Görselleştirme
        if ball_center and best_contour is not None:
            cv2.circle(frame, ball_center, 6, (0, 255, 0), -1)
            cv2.circle(frame, ball_center, 25, (0, 255, 0), 2)
            cv2.drawContours(frame, [best_contour], -1, (255, 0, 0), 2)

        # Maskeyi orijinal boyuta döndür
        white_mask = cv2.resize(white_mask, (frame.shape[1], frame.shape[0]))
        
        return frame, white_mask, ball_center

    def run(self):
        """Ana döngü - Hız optimizasyonlu"""
        cap = cv2.VideoCapture(self.camera_source)
        if not cap.isOpened():
            print(f"❌ Camera connection failed: {self.camera_source}")
            return False

        # Kamera ayarları optimize edildi
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 60)  # Daha yüksek FPS
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # Buffer boyutu minimize

        print(f"✅ Camera connected: {self.camera_source}")
        
        print("\n📸 İlk görüntü alınıyor...")
        time.sleep(0.5)
        ret, frame = cap.read()
        if not ret:
            print("❌ İlk görüntü alınamadı!")
            cap.release()
            return False

        platform_data = self.manual_platform_selection(frame)
        
        if platform_data is None:
            print("❌ Platform seçimi iptal edildi!")
            cap.release()
            cv2.destroyAllWindows()
            return False

        self.platform_center, self.platform_radius = platform_data
        self.calibrated = True
        print(f"✅ Platform kalibrasyonu tamamlandı!")
        print(f"   Merkez: {self.platform_center}")
        print(f"   Yarıçap: {self.platform_radius:.1f} piksel")

        stm32_connected = self.init_stm32_connection()
        self.running = True
        print("\n🎯 STM32 Ball Tracker Started (Q - quit)")
        print(f"⚡ Gönderim hızı: ~{int(1/self.send_interval)} Hz (maksimum)")

        frame_count = 0
        fps_start = time.time()

        try:
            while self.running:
                ret, frame = cap.read()
                if not ret:
                    continue

                display_frame = frame.copy()

                # Daire dışını maskeleme
                if self.calibrated:
                    mask = np.zeros_like(display_frame)
                    cv2.circle(mask, tuple(self.platform_center), int(self.platform_radius), 
                             (255, 255, 255), -1)
                    display_frame = cv2.bitwise_and(display_frame, mask)
                    
                    cv2.circle(display_frame, tuple(self.platform_center), 
                             int(self.platform_radius), (0, 255, 255), 2)
                    cv2.circle(display_frame, tuple(self.platform_center), 3, (0, 255, 255), -1)

                tracked_frame, mask_img, ball_pixel = self.detect_ball(display_frame)

                if ball_pixel and self.calibrated:
                    platform_coords = self.pixel_to_platform_coords(ball_pixel)
                    
                    if platform_coords:
                        coord_text = f"X:{platform_coords[0]:.2f}cm Y:{platform_coords[1]:.2f}cm"
                        cv2.putText(tracked_frame, coord_text, 
                                  (ball_pixel[0] + 30, ball_pixel[1] - 30),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                        
                        if stm32_connected:
                            self.send_coordinates_to_stm32(platform_coords)

                # FPS hesaplama
                frame_count += 1
                if frame_count % 30 == 0:
                    fps = 30 / (time.time() - fps_start)
                    fps_start = time.time()
                    cv2.putText(tracked_frame, f"FPS: {fps:.1f}", 
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

                cv2.imshow('STM32 Ball Tracker', tracked_frame)
                cv2.imshow('Ball Mask', mask_img)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    print("🛑 Stopped by user")
                    break

        except KeyboardInterrupt:
            print("\n⏹ Stopped with Ctrl+C")
        except Exception as e:
            print(f"❌ Error: {e}")
        finally:
            self.running = False
            cap.release()
            cv2.destroyAllWindows()
            if self.ser and self.ser.is_open:
                self.ser.close()
                print("🔌 STM32 connection closed")
            print("✅ System cleaned up")

        return True

def main():
    print("🤖 STM32 Ball Tracker - Hızlı Gönderim Modu")
    print("=" * 50)

    CAMERA_SOURCE = "http://192.168.1.172:8080/video"
    # CAMERA_SOURCE = 0

    STM32_PORT = 'COM5'
    BAUDRATE = 115200

    print(f"📱 Camera: {CAMERA_SOURCE}")
    print(f"🔌 STM32: {STM32_PORT} @ {BAUDRATE}")
    print(f"📐 Platform: Ø16 cm (dairesel)")
    print(f"⚡ Optimizasyon: 100Hz gönderim hedefi")
    print("-" * 50)

    tracker = STM32BallTracker(
        camera_source=CAMERA_SOURCE,
        stm32_port=STM32_PORT,
        baudrate=BAUDRATE
    )

    success = tracker.run()
    if success:
        print("✅ Program completed successfully")
    else:
        print("❌ Program ended with error")

if __name__ == "__main__":
    main()

🤖 STM32 Ball Tracker - Hızlı Gönderim Modu
📱 Camera: http://192.168.1.172:8080/video
🔌 STM32: COM5 @ 115200
📐 Platform: Ø16 cm (dairesel)
⚡ Optimizasyon: 100Hz gönderim hedefi
--------------------------------------------------
✅ Camera connected: http://192.168.1.172:8080/video

📸 İlk görüntü alınıyor...

🖱  MANUEL DAİRESEL KALIBRASYON
1. Platformun MERKEZİNE tıklayın
2. Platformun KENARINDA bir noktaya tıklayın
3. Seçim tamamlandıktan sonra SPACE basın
4. Yeniden başlamak için R basın
✓ Merkez: (148, 76)
✓ Kenar: (202, 117)
✓ Yarıçap: 67 piksel
✅ Daire seçildi! SPACE'e basın veya pencereyi kapatın...
✅ Platform kalibrasyonu tamamlandı!
   Merkez: [148, 76]
   Yarıçap: 67.8 piksel
✓ STM32 connected: COM5 @ 115200 baud

🎯 STM32 Ball Tracker Started (Q - quit)
⚡ Gönderim hızı: ~100 Hz (maksimum)
STM32 <- <011,089> [1000 Hz]
STM32 <- <016,084> [1000 Hz]
STM32 <- <037,084> [1000 Hz]
STM32 <- <044,084> [1000 Hz]
STM32 <- <045,084> [1000 Hz]
STM32 <- <046,084> [1000 Hz]
STM32 <- <046,085> [1